In [1]:
import os
import numpy as np
import pandas as pd

import torch
import torchvision.transforms as transforms
import torchvision.models as models

from PIL import Image
from tqdm import tqdm


In [2]:
BASE_DIR = r"C:\Users\Aiswarya\Desktop\Property_Valuation_XGBoost"
IMG_DIR = os.path.join(BASE_DIR, "images")
DATA_DIR = os.path.join(BASE_DIR, "data")


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

cnn = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
cnn.fc = torch.nn.Identity()   # remove classifier
cnn = cnn.to(device)
cnn.eval()


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\Aiswarya/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100%|███████████████████████████████████████████████████████████| 44.7M/44.7M [00:00<00:00, 78.9MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [4]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [5]:
visual_scores = []

image_files = sorted(os.listdir(IMG_DIR), key=lambda x: int(x.replace(".png", "")))

for img_name in tqdm(image_files):
    img_path = os.path.join(IMG_DIR, img_name)
    
    img = Image.open(img_path).convert("RGB")
    img = transform(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        feat = cnn(img).cpu().numpy().flatten()
    
    visual_index = feat.mean()   # 🔑 single visual feature
    visual_scores.append(visual_index)

visual_df = pd.DataFrame({
    "id": range(len(visual_scores)),
    "visual_index": visual_scores
})

visual_df.head()


100%|██████████████████████████████████████████████████████████████| 400/400 [00:19<00:00, 20.03it/s]


,id,visual_index
0,0,0.219879
1,1,0.283167
2,2,0.298053
3,3,0.250473
4,4,0.186615


In [6]:
visual_df.to_csv(os.path.join(DATA_DIR, "visual_features.csv"), index=False)
print("Visual features saved successfully.")


Visual features saved successfully.
